In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
!ls drive

MyDrive  Othercomputers


In [ ]:
%cd '/content/drive/MyDrive/GraphDTA-master'

/content/drive/MyDrive/GraphDTA-master


# New Section

In [ ]:
import torch

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=97f254f8b0367c23095600518e8624709c90a61df974a5a9ee215a4eeef2ac75
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [ ]:
!pip install torch torchvision numpy pandas scikit-learn tqdm matplotlib seaborn

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 55.4 MB/s eta 0:00:00


In [ ]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
import torch
import torch.nn as nn
from models.gat import GATNet
from models.gat_gcn import GAT_GATNet
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *
import matplotlib.pyplot as plt

In [ ]:

#import dataloader as dataloader
# training function at each epoch
def train(model, device, train_loader, optimizer, epoch):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, data.y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                            batch_idx * len(data.x),
                                            len(train_loader.dataset),
                                            100. * batch_idx / len(train_loader),
                                            loss.item()))

def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
    return total_labels.numpy().flatten(),total_preds.numpy().flatten()

In [ ]:
!pip install Tensorflow

In [ ]:
!pip install Scikit-learn
!pip install Matplotlib Seaborn
!pip install subword_nmt periodictable tensorflow_addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.5/752.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 18.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import glob
import json
import itertools
import periodictable as pt
import re
from itertools import chain
from operator import itemgetter
from subword_nmt.apply_bpe import BPE
import codecs

In [ ]:
!python '/content/drive/MyDrive/GraphDTA-master/create_data.py'

In [ ]:
!python training.py 0 2 0

cuda_name: cuda:0
Learning rate:  0.0005
Epochs:  2

running on  GAT_GATNet_davis
Pre-processed data found: data/processed/davis_train.pt, loading ...
Pre-processed data found: data/processed/davis_test.pt, loading ...
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Training on 25046 samples...
/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "
Train epoch: 1 [0/25046 (0%)]	Loss: 31.480980
Train epoch: 1 [333060/25046 (41%)]	Loss: 1.370630
Train epoch: 1 [646480/25046 (82%)]	Loss: 0.764220
Make prediction for 5010 samples...
rmse improved at epoch  1 ; best_mse,best_ci: 0.88426507 0.6203969778080272 GAT_GATNet davis
Training on 25046 samples...
Train epoc

In [ ]:
dataset = 'davis'
#kiba
modeling = [GAT_GATNet]
model_st = modeling[0].__name__

cuda_name = "cuda:0"
# if len(sys.argv)>3:
#     cuda_name = "cuda:" + str(int(sys.argv[3]))
# print('cuda_name:', cuda_name)

TRAIN_BATCH_SIZE = 512
TEST_BATCH_SIZE = 512
LR = 0.0005
LOG_INTERVAL = 20
NUM_EPOCHS = 2

print('Learning rate: ', LR)
print('Epochs: ', NUM_EPOCHS)

# Main program: iterate over different datasets
# for dataset in dataset:
print('\nrunning on ', model_st + '_' + dataset )
processed_data_file_train = 'data/processed/' + dataset + '_train.pt'
processed_data_file_test = 'data/processed/' + dataset + '_test.pt'
if ((not os.path.isfile(processed_data_file_train)) or \
 (not os.path.isfile(processed_data_file_test))):
    print('please run create_data.py to prepare data in pytorch format!')
else:
    train_data = TestbedDataset(root='data', dataset=dataset+'_train')
    test_data = TestbedDataset(root='data', dataset=dataset+'_test')

    # make data PyTorch mini-batch processing ready
    train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

# training the model
    device = torch.device(cuda_name if torch.cuda.is_available() else "cpu")
    #model = modeling
    for modeling in [GAT_GATNet]:
      model = modeling().to(device)
    loss_fn = nn.MSELoss()

    # model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    best_mse = 0.24929152
    best_ci =  0.8811023101087032
    best_epoch = 853
    model_file_name = 'model_' + model_st + '_' + dataset +  '.model'
    result_file_name = 'result_' + model_st + '_' + dataset +  '.csv'
    model.load_state_dict(torch.\
    load('/content/drive/MyDrive/GraphDTA-master/model_GAT_GATNet_davis.model',\
         map_location=torch.device('cpu'))
    )
     #map_location=torch.device('cpu'))) # load the state_dict on CPU
    # model.train()

    mse_list = []
    ci_list = []
    for epoch in range(NUM_EPOCHS):
      torch.cuda.empty_cache()
      train(model, device, train_loader, optimizer, epoch+1)
      G,P = predicting(model, device, test_loader)
      ret = [rmse(G,P),mse(G,P),pearson(G,P),spearman(G,P),ci(G,P)]


      mse_list.append(ret[1])
      ci_list.append(ret[-1])
      if ret[1]<best_mse:
          torch.save(model.state_dict(), \
                '/content/drive/MyDrive/GraphDTA-master/model_GAT_GATNet_davis.model')
          with open(result_file_name,'w') as f:
              f.write(','.join(map(str,ret)))
          best_epoch = epoch+1
          best_mse = ret[1]
          best_ci = ret[-1]


          print('rmse improved at epoch ', best_epoch, '; best_mse,best_ci:',\
                best_mse,best_ci,model_st,dataset)

      else:
          print(ret[1],'No improvement since epoch ', best_epoch,\
                '; best_mse,best_ci:', best_mse,best_ci,model_st,dataset)




Learning rate:  0.0005
Epochs:  2

running on  GAT_GATNet_davis
Pre-processed data found: data/processed/davis_train.pt, loading ...
Pre-processed data found: data/processed/davis_test.pt, loading ...


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Training on 25046 samples...


/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "


Train epoch: 1 [0/25046 (0%)]	Loss: 0.107014
Train epoch: 1 [329680/25046 (41%)]	Loss: 0.135768
Train epoch: 1 [657720/25046 (82%)]	Loss: 0.093029
Make prediction for 5010 samples...
rmse improved at epoch  1 ; best_mse,best_ci: 0.24873847 0.8773788627588337 GAT_GATNet davis
Training on 25046 samples...
Train epoch: 2 [0/25046 (0%)]	Loss: 0.072308
Train epoch: 2 [331360/25046 (41%)]	Loss: 0.098862
Train epoch: 2 [659080/25046 (82%)]	Loss: 0.069238
Make prediction for 5010 samples...
0.2679761 No improvement since epoch  1 ; best_mse,best_ci: 0.24873847 0.8773788627588337 GAT_GATNet davis
